In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Download TorchVision repo to use some files from
# references/detection
!pip install pycocotools --quiet
!git clone https://github.com/pytorch/vision.git
!git checkout v0.3.0

!cp vision/references/detection/utils.py ./
!cp vision/references/detection/transforms.py ./
!cp vision/references/detection/coco_eval.py ./
!cp vision/references/detection/engine.py ./
!cp vision/references/detection/coco_utils.py ./

Cloning into 'vision'...
remote: Enumerating objects: 531477, done.
remote: Counting objects: 100% (45087/45087), done.
remote: Compressing objects: 100% (2109/2109), done.
remote: Total 531477 (delta 43041), reused 44882 (delta 42905), pack-reused 486390
Receiving objects: 100% (531477/531477), 1022.60 MiB | 28.58 MiB/s, done.
Resolving deltas: 100% (496276/496276), done.
fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [3]:
# Basic python and ML Libraries
import os
import random
import numpy as np
import pandas as pd
# for ignoring warnings
import warnings
warnings.filterwarnings('ignore')

# We will be reading images using OpenCV
import cv2

# xml library for parsing xml files
from xml.etree import ElementTree as et

# matplotlib for visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# torchvision libraries
import torch
from torch.utils.data import ConcatDataset

from torch.utils.data import Subset
import torchvision
from torchvision import transforms as torchtrans  
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# these are the helper libraries imported.
from engine import train_one_epoch, evaluate
import utils
import transforms as T

# for image augmentations
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

In [4]:
mkdir -p /kaggle/working/train-inriaperson && cp -r /kaggle/input/inriaperson/Train/Annotations/* /kaggle/working/train-inriaperson

In [5]:
cp -r /kaggle/input/inriaperson/Train/JPEGImages/* /kaggle/working/train-inriaperson

In [6]:
# defining the files directory and testing directory
train_dir = '/kaggle/working/train-inriaperson'
test_dir = '/kaggle/input/citypersons-testselection/NewTest'

In [7]:
add_train_dir = '/kaggle/input/pedestriansyntheticdataset/train/train'

In [8]:
class ImagesDataset(torch.utils.data.Dataset):

    def __init__(self, files_dir, width, height, transforms=None):
        self.transforms = transforms
        self.files_dir = files_dir
        self.height = height
        self.width = width
            
        self.imgs = [image for image in sorted(os.listdir(files_dir)) if image.endswith(('.png', '.jpg'))]
        
        
        # classes: 0 index is reserved for background
        self.classes = ['background','pedestrian']

    def __getitem__(self, idx):

        img_name = self.imgs[idx]
        image_path = os.path.join(self.files_dir, img_name)

        # reading the images and converting them to correct size and color    
        img = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        img_res = cv2.resize(img_rgb, (self.width, self.height), cv2.INTER_AREA)
        # diving by 255
        img_res /= 255.0
        
        # annotation file
        annot_filename = img_name[:-4] + '.xml'
        annot_file_path = os.path.join(self.files_dir, annot_filename)
        
        boxes = []
        labels = []
        tree = et.parse(annot_file_path)
        root = tree.getroot()
        
        # cv2 image gives size as height x width
        wt = img.shape[1]
        ht = img.shape[0]
        
        # box coordinates for xml files are extracted and corrected for image size given
        for member in root.findall('object'):
            label_text = member.find('name').text.lower()
            if label_text in ['ped', 'pedestrian','person']:
                labels.append(self.classes.index('pedestrian')) 
            
            # bounding box
            xmin = int(member.find('bndbox').find('xmin').text)
            xmax = int(member.find('bndbox').find('xmax').text)
            
            ymin = int(member.find('bndbox').find('ymin').text)
            ymax = int(member.find('bndbox').find('ymax').text)
            
            
            xmin_corr = (xmin/wt)*self.width
            xmax_corr = (xmax/wt)*self.width
            ymin_corr = (ymin/ht)*self.height
            ymax_corr = (ymax/ht)*self.height
            
            boxes.append([xmin_corr, ymin_corr, xmax_corr, ymax_corr])
        
        # convert boxes into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        
        # getting the areas of the boxes
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        # suppose all instances are not crowd
        iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)
        
        labels = torch.as_tensor(labels, dtype=torch.int64)


        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["area"] = area
        target["iscrowd"] = iscrowd
        # image_id
        image_id = idx
        target["image_id"] = image_id
        
  


        if self.transforms:
            
            sample = self.transforms(image = img_res,
                                     bboxes = target['boxes'],
                                     labels = labels)
            
            img_res = sample['image']
            target['boxes'] = torch.Tensor(sample['bboxes'])
            
            
            
        return img_res, target

    def __len__(self):
        return len(self.imgs)

In [9]:

def get_object_detection_model(num_classes):

    # load a model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes) 

    return model

In [10]:
# Send train=True fro training transforms and False for val/test transforms
def get_transform(train):
    
    if train:
        return A.Compose([
            A.HorizontalFlip(0.5),
            A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=0.5),
            A.Rotate(limit=15, p=0.5),
            ToTensorV2(p=1.0)
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))
    else:
        return A.Compose([
                            ToTensorV2(p=1.0)
                        ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

In [11]:
# use our dataset and defined transformations
train_dataset = ImagesDataset(train_dir, 1920, 1080, transforms= get_transform(train=True))
add_train_dataset = ImagesDataset(add_train_dir, 1920, 1080, transforms= get_transform(train=True))
combined_dataset = ConcatDataset([train_dataset, add_train_dataset])

test_dataset = ImagesDataset(test_dir, 1920, 1080, transforms= get_transform(train=False))

# define training and validation data loaders
combined_data_loader_train = torch.utils.data.DataLoader(
    combined_dataset, batch_size=4, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    test_dataset, batch_size=4, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

In [12]:
# to train on gpu if selected.
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


num_classes = 2

# get the model using our helper function
model = get_object_detection_model(num_classes)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 142MB/s]


In [13]:
# training for 10 epochs
num_epochs = 10

for epoch in range(num_epochs):
    # training for one epoch
    train_one_epoch(model, optimizer, combined_data_loader_train, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

Epoch: [0]  [  0/227]  eta: 0:17:19  lr: 0.000027  loss: 1.0473 (1.0473)  loss_classifier: 0.7954 (0.7954)  loss_box_reg: 0.2224 (0.2224)  loss_objectness: 0.0120 (0.0120)  loss_rpn_box_reg: 0.0175 (0.0175)  time: 4.5776  data: 1.5016  max mem: 4799


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Epoch: [0]  [ 10/227]  eta: 0:03:39  lr: 0.000248  loss: 0.9360 (0.8761)  loss_classifier: 0.6801 (0.6305)  loss_box_reg: 0.2224 (0.2221)  loss_objectness: 0.0172 (0.0167)  loss_rpn_box_reg: 0.0063 (0.0068)  time: 1.0107  data: 0.1652  max mem: 4957


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Epoch: [0]  [ 20/227]  eta: 0:02:53  lr: 0.000469  loss: 0.6992 (0.7063)  loss_classifier: 0.2987 (0.4338)  loss_box_reg: 0.2259 (0.2471)  loss_objectness: 0.0176 (0.0189)  loss_rpn_box_reg: 0.0063 (0.0066)  time: 0.6508  data: 0.0309  max mem: 4958


...

Epoch: [9]  [200/227]  eta: 0:00:17  lr: 0.000005  loss: 0.1030 (0.1229)  loss_classifier: 0.0320 (0.0411)  loss_box_reg: 0.0629 (0.0778)  loss_objectness: 0.0006 (0.0014)  loss_rpn_box_reg: 0.0020 (0.0026)  time: 0.6520  data: 0.0348  max mem: 4961


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Epoch: [9]  [210/227]  eta: 0:00:11  lr: 0.000005  loss: 0.1106 (0.1231)  loss_classifier: 0.0365 (0.0411)  loss_box_reg: 0.0762 (0.0780)  loss_objectness: 0.0006 (0.0014)  loss_rpn_box_reg: 0.0020 (0.0026)  time: 0.6480  data: 0.0333  max mem: 4961


libpng warning: iCCP: known incorrect sRGB profile


Epoch: [9]  [220/227]  eta: 0:00:04  lr: 0.000005  loss: 0.1334 (0.1243)  loss_classifier: 0.0414 (0.0415)  loss_box_reg: 0.0836 (0.0788)  loss_objectness: 0.0006 (0.0014)  loss_rpn_box_reg: 0.0028 (0.0026)  time: 0.6443  data: 0.0316  max mem: 4961
Epoch: [9]  [226/227]  eta: 0:00:00  lr: 0.000005  loss: 0.1393 (0.1245)  loss_classifier: 0.0418 (0.0415)  loss_box_reg: 0.0901 (0.0790)  loss_objectness: 0.0003 (0.0014)  loss_rpn_box_reg: 0.0028 (0.0026)  time: 0.6328  data: 0.0295  max mem: 4961
Epoch: [9] Total time: 0:02:28 (0.6532 s / it)
creating index...
index created!
Test:  [ 0/19]  eta: 0:00:18  model_time: 0.2776 (0.2776)  evaluator_time: 0.0044 (0.0044)  time: 0.9925  data: 0.6046  max mem: 4961
Test:  [18/19]  eta: 0:00:00  model_time: 0.2525 (0.2510)  evaluator_time: 0.0044 (0.0066)  time: 0.3502  data: 0.0594  max mem: 4961
Test: Total time: 0:00:06 (0.3519 s / it)
Averaged stats: model_time: 0.2525 (0.2510)  evaluator_time: 0.0044 (0.0066)
Accumulating evaluation results..